## Importing Libraries

In [3]:
import numpy as np
import time
from scipy.stats import norm
from sklearn.linear_model import LinearRegression

## Euler Scheme Function 

In [4]:
def get_truncation_functions(scheme):
        if scheme == 1:  # Absorption scheme
            return (lambda x: np.maximum(x, 0),
                    lambda x: np.maximum(x, 0),
                    lambda x: np.maximum(x, 0))
        elif scheme == 2:  # Reflection scheme
            return (np.abs, np.abs, np.abs)
        elif scheme == 3:  # Partial truncation scheme
            return (lambda x: x, lambda x: x, lambda x: np.maximum(0, x))
        elif scheme == 4:  # Full truncation scheme
            return (lambda x: x, lambda x: np.maximum(0, x), lambda x: np.maximum(0, x))

## Function for Heston Model

In [17]:
def european_call_heston_model(S_0, K, T, r, V_0, theta, kappa, omega, rho, steps_per_year, num_paths, scheme):
    
    # Get the truncation functions
    f_1, f_2, f_3 = get_truncation_functions(scheme)
    
    # Calculate dt
    dt = 1 / steps_per_year
    
    # Initialize the matrices for V and ln(S)
    V_tilde = np.zeros((num_paths, int(T/dt)+1))
    V = np.zeros((num_paths, int(T/dt)+1))
    ln_S_current = np.zeros((num_paths, int(T/dt)+1))
    
    # Fill in the initial value
    V_tilde[:, 0] = V_0
    V[:, 0] = V_0
    ln_S_current[:, 0] = np.log(S_0)
    
    # Simulate dW_V
    dW_V = np.random.normal(0, np.sqrt(dt), (num_paths, int(T/dt)))

    # Simulate dZ
    dZ = np.random.normal(0, np.sqrt(dt), (num_paths, int(T/dt)))

    # Simulate Correlated Brownian Motion
    dW_S = rho * dW_V + np.sqrt(1 - rho**2) * dZ
    
    # Loop through each step and calculate V and ln(S) according to the Heston framework using Euler scheme
    for j in range(int(T/dt)):
        V_tilde[:, j+1] = (f_1(V_tilde[:, j]) 
                          - kappa * dt * (f_2(V_tilde[:, j]) - theta)
                          + omega * np.sqrt(f_3(V_tilde[:, j])) * dW_V[:, j])
        V[:, j+1] = f_3(V_tilde[:, j+1])
        ln_S_current[:, j+1] = (ln_S_current[:, j] + (r - 0.5 * V[:, j]) * dt 
                          + np.sqrt(V[:, j]) * dW_S[:, j])
    
    # Get the stock price at time T by converting the ln(S) process to S 
    S_T_current = np.exp(ln_S_current[:, -1])
    
    # Calculate the option price
    call_price_T_current = np.exp(-r * T) * np.mean(np.maximum(S_T_current - K, 0))
    
    return call_price_T_current

## Initializing the Variables

In [18]:
# Initialize the variables
S_0 = 100   # Spot price
K = 100     # Strike price
T = 5       # Time period (years)
r = 0.05    # Risk-neutral drift of the asset price
V_0 = 0.09  # Initial variance
theta = 0.09  # Long-term average variance
kappa = 2   # Speed of mean-reversion of the variance
omega = 1   # Volatility of volatility
rho = -0.3  # Instantaneous correlation coefficient
x_true = 34.9998 # True value of the call option

## Pricing Error Measures

In [19]:
# Pricing error measures
def calc_bias(x):
    return np.abs(np.mean(x) - x_true)

def calc_std(x):
    return np.sqrt(np.mean((x - np.mean(x))**2))

def calc_RMSE(x):
    return np.sqrt(calc_bias(x)**2 + calc_std(x)**2)

## Scheme Comparison

In [20]:
# Create the list of num_path
num_paths_list = [10000, 40000, 160000]

# Create the list of steps/year
steps_per_year_list = [20, 40, 80]

# Perform the monte-carlo simulation
num_iters = 100 # repeat each scheme 100 times
scheme_names = ['Absorption Scheme', 'Reflection Scheme', 'Partial Truncation Scheme', 'Full Truncation Scheme']

# Loop through every scheme
for scheme in range(1, 5):
    # Match the scheme name for printing out the results
    scheme_name = scheme_names[scheme-1]
    for i in range(3):
        num_paths = num_paths_list[i]
        steps_per_year = steps_per_year_list[i]
        print('------------------------------------------------')
        print(f'Method: {scheme_name} | Number of Paths: {num_paths} | Steps/year: {steps_per_year}')
        
        # Initialize the list of values
        call_price_T_list = np.zeros(num_iters)
        
        # Start the time
        start_time = time.time()

        # Repeat each scheme 100 times
        for j in range(num_iters):
            # Calculate the European call option prices
            call_price_T_list[j] = european_call_heston_model(
                S_0, K, T, r, V_0, theta, kappa, omega, rho, steps_per_year, num_paths, scheme)
        
        # Stop the time after repeating each scheme for 100 times
        elapsed_time = time.time() - start_time

        # Print out the elapsed time
        print(f'Time elapsed: {elapsed_time:.2f} seconds')
        
        # Print out the error measures and option price
        print(f'Bias: {calc_bias(call_price_T_list):.4f} | Std: {calc_std(call_price_T_list):.4f} | RMSE: {calc_RMSE(call_price_T_list):.4f}')
        print(f'Call Option Price: {np.mean(call_price_T_list):.4f} | True Value: {x_true}')


------------------------------------------------
Method: Absorption Scheme | Number of Paths: 10000 | Steps/year: 20
Time elapsed: 4.58 seconds
Bias: 2.1406 | Std: 0.6086 | RMSE: 2.2254
Call Option Price: 37.1404 | True Value: 34.9998
------------------------------------------------
Method: Absorption Scheme | Number of Paths: 40000 | Steps/year: 40
Time elapsed: 48.40 seconds
Bias: 1.5722 | Std: 0.3312 | RMSE: 1.6067
Call Option Price: 36.5720 | True Value: 34.9998
------------------------------------------------
Method: Absorption Scheme | Number of Paths: 160000 | Steps/year: 80
Time elapsed: 495.75 seconds
Bias: 1.1829 | Std: 0.1400 | RMSE: 1.1911
Call Option Price: 36.1827 | True Value: 34.9998
------------------------------------------------
Method: Reflection Scheme | Number of Paths: 10000 | Steps/year: 20
Time elapsed: 4.29 seconds
Bias: 4.5069 | Std: 0.8486 | RMSE: 4.5861
Call Option Price: 39.5067 | True Value: 34.9998
------------------------------------------------
Method:

## Delta and Gamma Function

In [21]:
def delta_gamma_calc(S_0, K, T, r, V_0, theta, kappa, omega, rho, steps_per_year, num_paths, scheme):
    
    # Get the truncation functions
    f_1, f_2, f_3 = get_truncation_functions(scheme)
    
    # Calculate dt
    dt = 1 / steps_per_year

    # Calculate dS
    dS = 0.01 * S_0
    
    # Initialize the matrices for V and ln(S)
    V_tilde = np.zeros((num_paths, int(T/dt)+1))
    V = np.zeros((num_paths, int(T/dt)+1))
    ln_S_current = np.zeros((num_paths, int(T/dt)+1))
    ln_S_before = np.zeros((num_paths, int(T/dt)+1))
    ln_S_after = np.zeros((num_paths, int(T/dt)+1))
    
    # Fill in the initial value
    V_tilde[:, 0] = V_0
    V[:, 0] = V_0
    ln_S_current[:, 0] = np.log(S_0)
    ln_S_before[:, 0] = np.log(S_0 - dS)
    ln_S_after[:, 0] = np.log(S_0 + dS)
    
    # Simulate dW_V
    dW_V = np.random.normal(0, np.sqrt(dt), (num_paths, int(T/dt)))

    # Simulate dZ
    dZ = np.random.normal(0, np.sqrt(dt), (num_paths, int(T/dt)))

    # Simulate Correlated Brownian Motion
    dW_S = rho * dW_V + np.sqrt(1 - rho**2) * dZ
    
    # Loop through each step and calculate V and ln(S) according to the Heston framework using Euler scheme
    for j in range(int(T/dt)):
        V_tilde[:, j+1] = (f_1(V_tilde[:, j]) 
                          - kappa * dt * (f_2(V_tilde[:, j]) - theta)
                          + omega * np.sqrt(f_3(V_tilde[:, j])) * dW_V[:, j])
        V[:, j+1] = f_3(V_tilde[:, j+1])
        ln_S_current[:, j+1] = (ln_S_current[:, j] + (r - 0.5 * V[:, j]) * dt 
                          + np.sqrt(V[:, j]) * dW_S[:, j])
        ln_S_before[:, j+1] = (ln_S_before[:, j] + (r - 0.5 * V[:, j]) * dt 
                          + np.sqrt(V[:, j]) * dW_S[:, j])
        ln_S_after[:, j+1] = (ln_S_after[:, j] + (r - 0.5 * V[:, j]) * dt 
                          + np.sqrt(V[:, j]) * dW_S[:, j])
    
    # Get the stock prices process at time T by converting the ln(S) process to S 
    S_T_current = np.exp(ln_S_current[:, -1])
    S_T_before = np.exp(ln_S_before[:, -1])
    S_T_after = np.exp(ln_S_after[:, -1])
    
    # Calculate the option prices
    call_price_T_currrent = np.exp(-r * T) * np.mean(np.maximum(S_T_current - K, 0))
    call_price_T_before = np.exp(-r * T) * np.mean(np.maximum(S_T_before - K, 0))
    call_price_T_after = np.exp(-r * T) * np.mean(np.maximum(S_T_after - K, 0))
    
    # Calculate delta and gamma using finite difference method
    delta = (call_price_T_after - call_price_T_before) / (2 * dS)
    gamma = (call_price_T_after - 2 * call_price_T_currrent + call_price_T_before) / (dS ** 2)
    
    return call_price_T_currrent, delta, gamma

## Initializing Variables

In [22]:
# Initialize the variables
# The previous variables are the same
sigma = 0.3 # Sigma
num_paths = 100000 # 100000 path
steps_per_year = 50 # 50 steps/year
scheme = 4 # Full truncation scheme

## Calculating Delta and Gamma using the Black-Scholes model

In [23]:
# Calculate Delta and Gamma by using the Black-Scholes model
def black_scholes_greeks(S, K, r, T, sigma):
    d1 = (np.log(S / K) + (r + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
    delta = norm.cdf(d1)
    gamma = norm.pdf(d1) / (S * sigma * np.sqrt(T))
    return delta, gamma

## Calculating Delta and Gamma using Finite Difference Method

In [24]:
# Print out the Delta and Gamma from the Black-Scholes model
Delta_BS, Gamma_BS = black_scholes_greeks(S_0, K, r, T, sigma)
print(f'Delta_BS: {Delta_BS:.4f}')
print(f'Gamma_BS: {Gamma_BS:.4f}')

# Perform the monte-carlo simulation
num_iters = 100 # repeat each scheme 100 times

scheme_name = 'Full Truncation Scheme'
print('------------------------------------------------')
print(f'Method: {scheme_name} | Number of Paths: {num_paths} | Steps/year: {steps_per_year}')
        
# Initialize the list of values
call_price_T_list = np.zeros(num_iters)
Delta_list = np.zeros(num_iters)
Gamma_list = np.zeros(num_iters)
        
# Start the time
start_time = time.time()

# Repeat each scheme 100 times
for j in range(num_iters):
    # Calculate the European call option prices, delta, and gamma
    call_price_T_list[j], Delta_list[j], Gamma_list[j] = delta_gamma_calc(
        S_0, K, T, r, V_0, theta, kappa, omega, rho, steps_per_year, num_paths, scheme)
        
# Stop the time after repeating each scheme for 100 times
elapsed_time = time.time() - start_time

# Print out the elapsed time
print(f'Time elapsed: {elapsed_time:.2f} seconds')
        
# Print out the error measures and option price
print(f'Bias: {calc_bias(call_price_T_list):.4f} | Std: {calc_std(call_price_T_list):.4f} | RMSE: {calc_RMSE(call_price_T_list):.4f}')
print(f'Call Option Price: {np.mean(call_price_T_list):.4f} | True Value: {x_true} | Delta: {np.mean(Delta_list):.4f} | Gamma: {np.mean(Gamma_list):.4f}')

Delta_BS: 0.7606
Gamma_BS: 0.0046
------------------------------------------------
Method: Full Truncation Scheme | Number of Paths: 100000 | Steps/year: 50
Time elapsed: 275.69 seconds
Bias: 0.0386 | Std: 0.1839 | RMSE: 0.1879
Call Option Price: 35.0384 | True Value: 34.9998 | Delta: 0.7964 | Gamma: 0.0047


## Scheme Comparison (Omega=0.3)

In [25]:
# Change the omega to 0.3
omega = 0.3

# Create the list of num_path
num_paths_list = [10000, 40000, 160000]

# Create the list of steps/year
steps_per_year_list = [20, 40, 80]

# Perform the monte-carlo simulation
num_iters = 100 # repeat each scheme 100 times
scheme_names = ['Absorption Scheme', 'Reflection Scheme', 'Partial Truncation Scheme', 'Full Truncation Scheme']

# Loop through every scheme
for scheme in range(1, 5):
    # Match the scheme name for printing out the results
    scheme_name = scheme_names[scheme-1]
    for i in range(3):
        num_paths = num_paths_list[i]
        steps_per_year = steps_per_year_list[i]
        print('------------------------------------------------')
        print(f'Method: {scheme_name} | Number of Paths: {num_paths} | Steps/year: {steps_per_year}')
        
        # Initialize the list of values
        call_price_T_list = np.zeros(num_iters)
        
        # Start the time
        start_time = time.time()

        # Repeat each scheme 100 times
        for j in range(num_iters):
            # Calculate the European call option prices
            call_price_T_list[j] = european_call_heston_model(
                S_0, K, T, r, V_0, theta, kappa, omega, rho, steps_per_year, num_paths, scheme)
        
        # Stop the time after repeating each scheme for 100 times
        elapsed_time = time.time() - start_time

        # Print out the elapsed time
        print(f'Time elapsed: {elapsed_time:.2f} seconds')
        
        # Print out the error measures and option price
        print(f'Bias: {calc_bias(call_price_T_list):.4f} | Std: {calc_std(call_price_T_list):.4f} | RMSE: {calc_RMSE(call_price_T_list):.4f}')
        print(f'Call Option Price: {np.mean(call_price_T_list):.4f} | True Value: {x_true}')

------------------------------------------------
Method: Absorption Scheme | Number of Paths: 10000 | Steps/year: 20
Time elapsed: 4.16 seconds
Bias: 0.8636 | Std: 0.5916 | RMSE: 1.0468
Call Option Price: 35.8634 | True Value: 34.9998
------------------------------------------------
Method: Absorption Scheme | Number of Paths: 40000 | Steps/year: 40
Time elapsed: 43.65 seconds
Bias: 0.9320 | Std: 0.3241 | RMSE: 0.9868
Call Option Price: 35.9318 | True Value: 34.9998
------------------------------------------------
Method: Absorption Scheme | Number of Paths: 160000 | Steps/year: 80
Time elapsed: 503.85 seconds
Bias: 0.8729 | Std: 0.1426 | RMSE: 0.8845
Call Option Price: 35.8727 | True Value: 34.9998
------------------------------------------------
Method: Reflection Scheme | Number of Paths: 10000 | Steps/year: 20
Time elapsed: 3.95 seconds
Bias: 0.8108 | Std: 0.6067 | RMSE: 1.0126
Call Option Price: 35.8106 | True Value: 34.9998
------------------------------------------------
Method:

## Calculating European Put Option with Full Truncation under Heston Model

In [5]:
def european_put_heston_model(S_0, K, T, r, V_0, theta, kappa, omega, rho, steps_per_year, num_paths, scheme):
    
    # Get the truncation functions
    f_1, f_2, f_3 = get_truncation_functions(scheme)
    
    # Calculate dt
    dt = 1 / steps_per_year
    
    # Initialize the matrices for V and ln(S)
    V_tilde = np.zeros((num_paths, int(T/dt)+1))
    V = np.zeros((num_paths, int(T/dt)+1))
    ln_S_current = np.zeros((num_paths, int(T/dt)+1))
    
    # Fill in the initial value
    V_tilde[:, 0] = V_0
    V[:, 0] = V_0
    ln_S_current[:, 0] = np.log(S_0)
    
    # Simulate dW_V
    dW_V = np.random.normal(0, np.sqrt(dt), (num_paths, int(T/dt)))

    # Simulate dZ
    dZ = np.random.normal(0, np.sqrt(dt), (num_paths, int(T/dt)))

    # Simulate Correlated Brownian Motion
    dW_S = rho * dW_V + np.sqrt(1 - rho**2) * dZ
    
    # Loop through each step and calculate V and ln(S) according to the Heston framework using Euler scheme
    for j in range(int(T/dt)):
        V_tilde[:, j+1] = (f_1(V_tilde[:, j]) 
                          - kappa * dt * (f_2(V_tilde[:, j]) - theta)
                          + omega * np.sqrt(f_3(V_tilde[:, j])) * dW_V[:, j])
        V[:, j+1] = f_3(V_tilde[:, j+1])
        ln_S_current[:, j+1] = (ln_S_current[:, j] + (r - 0.5 * V[:, j]) * dt 
                          + np.sqrt(V[:, j]) * dW_S[:, j])
    
    # Get the stock price at time T by converting the ln(S) process to S 
    S_T_current = np.exp(ln_S_current[:, -1])
    
    # Calculate the option price
    put_price_T_current = np.exp(-r * T) * np.mean(np.maximum(K - S_T_current, 0))
    
    return put_price_T_current

In [14]:
# Initialize the variables
S_0 = 100   # Spot price
K = 100     # Strike price
T = 5       # Time period (years)
r = 0.05    # Risk-neutral drift of the asset price
V_0 = 0.09  # Initial variance
theta = 0.09  # Long-term average variance
kappa = 2   # Speed of mean-reversion of the variance
omega = 1   # Volatility of volatility
rho = -0.3  # Instantaneous correlation coefficient
steps_per_year = 50 # 50 steps/year
num_paths = 10000 # 10000 simulated paths

print(f'The European put option price using Full Truncation scheme under Heston Model is \
{european_put_heston_model(S_0, K, T, r, V_0, theta, kappa, omega, rho, steps_per_year, num_paths, scheme=4)}')

The European put option price using Full Truncation scheme under Heston Model is 12.714589758735851


## Calculating American Put Option (LSMC) with Full Truncation under Heston Model

In [28]:
def american_put_heston_model(S_0, K, T, r, V_0, theta, kappa, omega, rho, steps_per_year, num_paths, scheme):
    
    # Calculate number of steps
    num_steps = int(T * steps_per_year)

    # Get the truncation functions    
    f_1, f_2, f_3 = get_truncation_functions(scheme)
    
    # Calculate dt
    dt = 1 / steps_per_year
    
    # Initialize the matrices for V and ln(S)
    V_tilde = np.zeros((num_paths, int(T/dt)+1))
    V = np.zeros((num_paths, int(T/dt)+1))
    ln_S_current = np.zeros((num_paths, int(T/dt)+1))
    
    # Fill in the initial value
    V_tilde[:, 0] = V_0
    V[:, 0] = V_0
    ln_S_current[:, 0] = np.log(S_0)
    
    # Simulate dW_V
    dW_V = np.random.normal(0, np.sqrt(dt), (num_paths, int(T/dt)))

    # Simulate dZ
    dZ = np.random.normal(0, np.sqrt(dt), (num_paths, int(T/dt)))

    # Simulate Correlated Brownian Motion
    dW_S = rho * dW_V + np.sqrt(1 - rho**2) * dZ
    
    # Loop through each step and calculate V and ln(S) according to the Heston framework using Euler scheme
    for j in range(int(T/dt)):
        V_tilde[:, j+1] = (f_1(V_tilde[:, j]) 
                          - kappa * dt * (f_2(V_tilde[:, j]) - theta)
                          + omega * np.sqrt(f_3(V_tilde[:, j])) * dW_V[:, j])
        V[:, j+1] = f_3(V_tilde[:, j+1])
        ln_S_current[:, j+1] = (ln_S_current[:, j] + (r - 0.5 * V[:, j]) * dt 
                          + np.sqrt(V[:, j]) * dW_S[:, j])
    
    # Get the stock prices by converting the ln(S) process to S 
    S_T_current = np.exp(ln_S_current)
    
    # Calculate the payoff
    payoff = np.maximum(K - S_T_current, 0)

    # Initialize the value matrix as the payoff matrix
    value = np.copy(payoff)
    
    # Loop through steps
    for t in range(num_steps - 1, 0, -1):

        # Get the stock prices and variances at time t
        S_t = S_T_current[:, t]
        V_t = V[:, t]

        # Fit the values
        features = np.column_stack((np.ones_like(S_t), S_t, S_t**2, V_t, V_t**2, S_t * V_t))
        model = LinearRegression().fit(features, np.exp(-r * dt) * value[:, t+1])

        # Predict the continuation value
        continuation_value = model.predict(features)

        # Compare the continuation value to the payoff
        value[:, t] = np.where(payoff[:, t] > continuation_value, payoff[:, t], np.exp(-r * dt) * value[:, t+1])

    # Calculate the American put option price
    price = np.mean(np.exp(-r * dt) * value)
    return price

In [29]:
# Initialize the variables
S_0 = 100   # Spot price
K = 100     # Strike price
T = 5       # Time period (years)
r = 0.05    # Risk-neutral drift of the asset price
V_0 = 0.09  # Initial variance
theta = 0.09  # Long-term average variance
kappa = 2   # Speed of mean-reversion of the variance
omega = 1   # Volatility of volatility
rho = -0.3  # Instantaneous correlation coefficient
steps_per_year = 50 # 50 steps/year
num_paths = 10000 # 10000 simulated paths

print(f'The American put option price using LSMC with Full Truncation under Heston Model is \
{american_put_heston_model(S_0, K, T, r, V_0, theta, kappa, omega, rho, steps_per_year, num_paths, scheme=4)}')

The American put option price using LSMC with Full Truncation under Heston Model is 16.13530526717778
